# TMDB -- The Movie Database

<b> Dataset </b> <br>
-- 26 million ratings . 270,000 users . 45,000 movies

<b>Pipeline</b> <br>
-- Import Libraries

<b> Reference:</b> <br>
Dataset: https://www.kaggle.com/rounakbanik/the-movies-dataset <br>
EDA: https://www.kaggle.com/rounakbanik/the-story-of-film/ <br>
Recommendation System: https://www.kaggle.com/rounakbanik/movie-recommender-systems


# Import Libraries

In [10]:
from IPython.display import Image, HTML
import json
import datetime
import ast

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from scipy import stats
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier, XGBRegressor

from wordcloud import WordCloud, STOPWORDS

import plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import warnings
warnings.filterwarnings('ignore')
import chart_studio
chart_studio.tools.set_credentials_file(username='rounakbanik', api_key='xTLaHBy9MVv5szF4Pwan')

sns.set_style('whitegrid')
sns.set(font_scale=1.25)
pd.set_option('display.max_colwidth', 50)

# 1) EDA

## DATA OBSERVATION

FEATURES

-- adult: X-Rated or Adult <br>
-- belongs_to_collection: Breakdown information on movie series<br>
-- budget: Budget in USD$ <br>
-- genres: Genres <br>
-- homepage: The official Homepage <br>
-- id: id <br>
-- imdb_id: IMDB ID <br>
-- original_language: Language which the movie was originally shot in <br>
-- original_title: Original title of the movie <br>
-- overview: Brief summary of movie<br>
-- popularity: TMDB popularity score <br>
-- poster_path: Poster link <br>
-- production_companies: Production companies <br>
-- release_date: Theatrical Release Date <br>
-- revenue: Total revenue in USD$ <br>
-- runtime: Total runtime in minutes <br>
-- spoken_languages: Spoken languages <br>
-- status: Status such as Released, To Be Released, Announced, etc <br>
-- tagline: Tagline <br>
-- title: The official title <br>
-- video: Indicates if there is a video present of the movie with TMDB <br>
-- vote_average: The average rating of the movie <br>
-- vote_count: The number of votes by uses, counted by TMDB <br>

In [15]:
TMDBdf = pd.read_csv("/Users/wonghuiee/AI_DataScience/Xccelerate/04MLApplications/TMDB/Dataset/movies_metadata.csv")
TMDBdf.head().transpose()

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col..."
budget,30000000,65000000,0,16000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 35, 'name': 'Comedy'}]"
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...


There are 45,466 movies with 24 features. Most features have very few NaN values except homepage and tagline. 

In [19]:
TMDBdf.shape

(45466, 24)

In [18]:
TMDBdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

## DATA WRANGLING